In [1]:
import mlrose_hiive as mlrose
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import time
import random as rn
import os
import time
import sys
%matplotlib inline

In [2]:
"""
Setting up seed values for reproducability
"""
starting_seed = 1234
seed_values = []

for i in range(0, 5):
    seed_values.append(starting_seed + i)

# np.random.seed(seed)
# rn.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)

# First run

Trying the different algorithms on the optimization problem with differing problem sizes before messing with the hyperparameters and the different iterations amounts.

In [9]:
"""
Main custom runner for the KnapSack optimization problem.

For this run, we just want to tune the hyperparameters to a good amount for the 
optimizations being done later.
"""

# Hyperparameters All
max_weight_pct = 0.6

# Hyperparamters RHC
max_attempts_rhc = 100
max_iters_rhc = np.inf
restarts = 25

# Hyperparameters SA
schedule = mlrose.ExpDecay()
max_attempts_sa = 100
max_iters_sa = np.inf

# Hyperparameters GA
pop_size_ga = 200
mutation_prob = 0.1
max_attempts_ga = 100
max_iters_ga = np.inf

# Hyperparameters MIMIC
pop_size_mimic = 400
keep_pct = 0.8
max_attempts_mimic = 100
max_iters_mimic = np.inf

problem_sizes = [1 * i for i in range(0, 160, 10)]
problem_sizes.pop(0)

rhc_data_problem_sizes = []
sa_data_problem_sizes = []
ga_data_problem_sizes = []
mimic_data_problem_sizes = []

# Want to loop across the different problem sizes
for problem_size in problem_sizes:
    rhc_data_avg = []
    sa_data_avg = []
    ga_data_avg = []
    mimic_data_avg = []
    
    print(f"Running problem size {problem_size}!")
    # Want to loop across the random seeds to set seed values for averaging
    for seed in seed_values:
        
        # Setting the seed values for random calls
        np.random.seed(seed)
        rn.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        
        # Generate the random initial state
        dist_list = set()  
        while len(dist_list) < problem_size:
            x = np.random.randint(1, int(problem_size * 2.5))
            y = np.random.randint(1, int(problem_size * 2.5))
            dist_list.add((x, y))

        dist_list = list(dist_list)
        
        # Create fitness function
        fitness = mlrose.TravellingSales(coords = dist_list)
        # Create Optimization Problem Object
        problem = mlrose.TSPOpt(
            length = problem_size,
            fitness_fn = fitness,
            maximize = False
        )
        
        problem.set_mimic_fast_mode(True)
        
        # Running the actual RHO algorithm while measuring times
        start = time.time()
        
        best_state, best_fitness, best_curve = mlrose.random_hill_climb(
            problem=problem,
            max_attempts=max_attempts_rhc,
            max_iters=max_iters_rhc,
            restarts=restarts,
            curve=True,
            random_state=seed
        )
        
        end = time.time()
        
        rhc_data_avg.append(best_fitness)
        
        problem.reset()
        
        # Running the actual SA algorithm while measuring times
        start = time.time()
        
        best_state, best_fitness, best_curve = mlrose.simulated_annealing(
            problem=problem,
            schedule=schedule,
            max_attempts=max_attempts_sa,
            max_iters=max_iters_sa,
            curve=True,
            random_state=seed
        )
        
        end = time.time()
        sa_data_avg.append(best_fitness)
        
        problem.reset()
        
        # Running the actual GA algorithm while measuring times
        start = time.time()
        
        best_state, best_fitness, best_curve = mlrose.genetic_alg(
            problem=problem,
            max_attempts=max_attempts_ga,
            max_iters=max_iters_ga,
            pop_size=pop_size_ga,
            mutation_prob=mutation_prob,
            curve=True,
            random_state=seed
        )
        
        end = time.time()
        ga_data_avg.append(best_fitness)
        
        problem.reset()
        
         # Running the actual MIMIC algorithm while measuring times
        start = time.time()
        
        best_state, best_fitness, best_curve = mlrose.mimic(
            problem=problem,
            max_attempts=max_attempts_mimic,
            max_iters=max_iters_mimic,
            pop_size=pop_size_mimic,
            keep_pct=keep_pct,
            curve=True,
            random_state=seed
        )
        
        end = time.time()
        mimic_data_avg.append(best_fitness)
        
        problem.reset()
    
    rhc_avg = np.average(rhc_data_avg)
    print(f"RHC best avg fitness {rhc_avg}")
    rhc_data_problem_sizes.append(rhc_avg)
    
    sa_avg = np.average(sa_data_avg)
    print(f"SA best avg fitness {sa_avg}")
    sa_data_problem_sizes.append(sa_avg)
    
    ga_avg = np.average(ga_data_avg)
    print(f"GA best avg fitness {ga_avg}")
    ga_data_problem_sizes.append(ga_avg)
    
    mimic_avg = np.average(mimic_data_avg)
    print(f"MIMIC best avg fitness {mimic_avg}")
    mimic_data_problem_sizes.append(mimic_avg)
    
    print("====================================================")


Running problem size 10!
RHC best avg fitness 63.6416194103257
SA best avg fitness 65.16641550927878
GA best avg fitness 63.64161941032569
MIMIC best avg fitness 66.25250013144284
Running problem size 20!
RHC best avg fitness 188.3879831166231
SA best avg fitness 238.95451832791346
GA best avg fitness 193.63623744626722
MIMIC best avg fitness 277.269026395365
Running problem size 30!


KeyboardInterrupt: 